In [1]:
# pip install xlrd

import sys 
import os
import pandas as pd
import re

In [2]:
project_path = os.path.dirname(os.path.abspath('..'))
if project_path not in sys.path:
    sys.path.append(project_path)
sys.path

['C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner\\src\\services',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\python311.zip',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\DLLs',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3',
 '',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner']

In [3]:
from config.config import config

In [4]:
# excel_file.xlsx если генерируем из "unique_comments_to_excel.py"

# df = pd.read_excel("excel_file.xlsx")
# df.shape

# generated_services.xlsx если генерируем из service_generator.ipynb (для новых услуг)

df = pd.read_excel("generated_services.xlsx")
df.shape

(238, 3)

In [7]:
df.head(2)

,service,comment,code
0,Терминальная обработка,Терминальная обработка по приёму (40' контейне...,000000421
1,Линейный фрахтовый сбор (THC),Линейный фрахтовый сбор (ТНС),000000420


In [8]:
if 'id' in df.columns:
    del df['id']

df.head(2)

,service,comment,code
0,Терминальная обработка,Терминальная обработка по приёму (40' контейне...,000000421
1,Линейный фрахтовый сбор (THC),Линейный фрахтовый сбор (ТНС),000000420


In [9]:
# преобразование в строку, замена переноса строк на пробел, удаление повторяющихся пробелов 

for col in df.columns:
    df[col] = df[col].apply(lambda x: str(x).replace('\n', ' ').strip())
    df[col] = df[col].apply(lambda x: re.sub(r' {2,}', ' ', x).strip())

In [10]:
df.describe()

,service,comment,code
count,238,238,238
unique,210,236,211
top,Определение содержания остаточных количеств пе...,Оформление документации: Оформление протокола ...,000000593
freq,3,2,3


In [11]:
df.shape

(238, 3)

### Экспорт списка наиболе часто встречающихся "Услуг" 

In [ ]:
service_counts = df['service'].value_counts()

filtered_services = service_counts[service_counts > 3]
filtered_services

In [ ]:
# import json 

# file = os.path.join(config['BASE_DIR'], 'config', 'freq_services.json')
# with open(file, 'w', encoding='utf-8') as f:
#     json.dump(filtered_services.index.to_list(), f, ensure_ascii=False, indent=4)

### Удаление транспортных позиций

In [ ]:
def remove_special_characters(text):
    response = re.sub(r'[^\w\s]', ' ', text.lower())
    return re.sub(r'(\s{2,}|\n)', ' ', response)

excluded = ['счета авто грузятся по реестрам', 'услуги грузятся автоматически', 'Счета авто. ']
excluded = [remove_special_characters(x) for x in excluded]
print(excluded)

In [ ]:
df['comment_cleaned'] = df['comment'].apply(remove_special_characters)
df = df[~df['comment_cleaned'].str.strip().isin(excluded)]
df = df.drop('comment_cleaned', axis=1)
df.shape

In [ ]:
df.describe()

### Удаление позиций "услуга русс. яз для акта"

In [ ]:
excluded = ['услуга русс.яз для акта']
excluded = [remove_special_characters(x) for x in excluded]
print(excluded)

In [ ]:
df['comment_cleaned'] = df['comment'].apply(remove_special_characters)
df = df[~df['comment_cleaned'].str.strip().isin(excluded)]
df = df.drop('comment_cleaned', axis=1)
df.shape

### Количество различных комментариев в одной услуге

In [ ]:
uniq_services = df.groupby(['service']).agg(num_comments=('comment', 'count'), uniq_comments=('comment', 'nunique')).reset_index()
uniq_services.tail()

In [ ]:
uniq_services.query("uniq_comments > 1")

### Количество различных услуг в одном комментарии

In [ ]:
uniq_comments = df.groupby(['comment']).agg(num_services=('service', 'count'), uniq_services=('service', 'nunique')).reset_index()
uniq_comments.tail()

In [ ]:
uniq_comments.query("uniq_services > 1")

### Услуги с двумя и более кодами

In [ ]:
service_code_counts = df.groupby('service')['code'].nunique()

services_with_multiple_codes = service_code_counts[service_code_counts > 1]
services_with_multiple_codes

In [ ]:
grouped = df[df['service'].isin(services_with_multiple_codes.index)].groupby(['service', 'code']).size().reset_index(name='count')
display(grouped)

idx = grouped.groupby('service')['count'].idxmin()
min_count_code = grouped.loc[idx]
display(min_count_code)

In [ ]:
df_ = df[~df['code'].isin(min_count_code.code)]

In [ ]:
df_.describe()

### Группировка

In [12]:
# old

# grouped_comment = df.groupby(['comment']).agg(
#     service_list=('service', lambda x: list(dict.fromkeys(x))),
#     codelist = ('code', lambda x: list(dict.fromkeys(x)))
# ).reset_index()
# grouped_comment['len_service'] = grouped_comment['service_list'].apply(len)
# grouped_comment['id'] = list(range(1, grouped_comment.shape[0]+1))

In [13]:
def custom_agg(group):
    service = list(group['service'])
    code = list(group['code'])
    service_code_zip = list(dict.fromkeys(zip(service, code)))
    service_list, code_list = zip(*service_code_zip)

    return pd.Series({'service_list': list(service_list), 'code_list': list(code_list)})

grouped_comment = df.groupby(['comment']).apply(custom_agg).reset_index()
grouped_comment['len_service'] = grouped_comment['service_list'].apply(len)
grouped_comment['id'] = list(range(grouped_comment.shape[0]))

grouped_comment.head()

,comment,service_list,code_list,len_service,id
0,3410,[Выдача контейнера: Контейнер ИСО 20'/40' груж...,[ТК-009528],1,0
1,AMENDMENT FEE,[AMENDMENT FEE],[ТК-008738],1,1
2,Additional charges,[Additional charges],[ТК-009557],1,2
3,Agency Recovery Fee,[Agency Recovery Fee (фрахт)],[ТК-008991],1,3
4,B/L SURRENDER FEE,[B/L SURRENDER FEE (фрахт)],[ТК-008964],1,4


In [14]:
error = "Длина service_list всегда должна быть равна длине code_list"

assert grouped_comment[grouped_comment['service_list'].apply(len) != grouped_comment['code_list'].apply(len)].size == 0, error

In [15]:
z = grouped_comment[grouped_comment["comment"] == "Транспортно-экспедиторские услуги ТрансКонтейнер"]
display(z)
display(z['service_list'].to_list())
display(z['code_list'].to_list())
display(len(z['service_list'].to_list()) == len(z['code_list'].to_list()))

,comment,service_list,code_list,len_service,id


[]

[]

True

In [16]:
grouped_comment.shape

(236, 5)

In [17]:
# Функция для создания строки формата 'service_list#code_list#'

def create_service_code_string(row):
    return [f"{s}#{c}#" for s, c in zip(row['service_list'], row['code_list'])]

# Применение функции ко всем строкам с помощью apply
grouped_comment['service_code'] = grouped_comment.apply(create_service_code_string, axis=1)
grouped_comment.tail()

,comment,service_list,code_list,len_service,id,service_code
231,охрана и сопровождение груза на поезде,[охрана и сопровождение груза],[Т2760],1,231,[охрана и сопровождение груза#Т2760#]
232,проведение испытаний: Свежие,[проведение испытаний продукции],[Т0666],1,232,[проведение испытаний продукции#Т0666#]
233,установка и снятие генератора по договору,[установка и снятие генератора],[ТК-006287],1,233,[установка и снятие генератора#ТК-006287#]
234,хранение 20фут. контейнера импорт с 9-х суток,[Хранение],[ТК-000473],1,234,[Хранение#ТК-000473#]
235,хранение контейнера импорт,[Хранение контейнера],[Т0464],1,235,[Хранение контейнера#Т0464#]


In [18]:
result_df = grouped_comment[['id', 'comment', 'service_code']]
result_df.tail()

,id,comment,service_code
231,231,охрана и сопровождение груза на поезде,[охрана и сопровождение груза#Т2760#]
232,232,проведение испытаний: Свежие,[проведение испытаний продукции#Т0666#]
233,233,установка и снятие генератора по договору,[установка и снятие генератора#ТК-006287#]
234,234,хранение 20фут. контейнера импорт с 9-х суток,[Хранение#ТК-000473#]
235,235,хранение контейнера импорт,[Хранение контейнера#Т0464#]


In [21]:
result_df.to_json("new_unique_comments.json", orient='records', force_ascii=False, indent=4)